## Düzgün görmek için vs code'a jupyter paketini yüklemeniz lazım

In [38]:
import pyomo.environ as pyo
import pandas as pd

In [39]:
#!pip install openpyxl

directory="Scenarios\ProjectPart1-Scenario3.xlsx"
#pandas depens on this package to read excel files
orders=pd.read_excel(directory,sheet_name="Orders")
orders.head()

,Order ID,Product Type,Demand Amount,Due Date,Earliness Penalty
0,1,P2,130,6,11
1,1,P4,150,6,11
2,1,P6,140,6,11
3,1,P7,140,6,11
4,1,P9,175,6,11


In [40]:
trucks=pd.read_excel(directory,sheet_name="Vehicles")
trucks

,Vehicle ID,Vehicle Type,Capacity for pallet type 1,Capacity for pallet type 2,Fixed Cost (c_k),Variable Cost (c'_k)
0,1,1,22,33,100,150
1,2,2,12,18,80,120
2,3,3,6,8,50,75
3,4,1,22,33,100,150
4,5,2,12,18,80,120
5,6,3,6,8,50,75
6,7,1,22,33,100,150
7,8,2,12,18,80,120
8,9,3,6,8,50,75


In [41]:
pallets=pd.read_excel(directory,sheet_name="Pallets")
pallets

,Pallet ID,Product Type,Amount,Pallet Size,Release Day
0,1,P1,95,2,1
1,2,P1,75,2,1
2,3,P1,60,2,1
3,4,P1,90,2,1
4,5,P3,55,2,1
...,...,...,...,...,...
431,432,P8,115,1,7
432,433,P9,70,2,7
433,434,P9,95,2,7
434,435,P10,60,2,7


In [43]:
extraParam=pd.read_excel(directory,sheet_name="Parameters")
extraParam

,Parameter,Value
0,Planning Horizon (T),7
1,Max. trips per period,3
2,Max. pallets in area,15


In [44]:
# --------------------------------------------------------
# 1) Create the model
# --------------------------------------------------------
model = pyo.ConcreteModel()


# --------------------------------------------------------
# 2) Define Sets
# --------------------------------------------------------
model.I = pyo.Set(initialize=range(1, len(pallets)+1), doc="Set of pallets")
model.K = pyo.Set(initialize=trucks["Vehicle Type"].unique().tolist(), doc="Set of vehicle types")

# Don't forget to change the indexes if there is a change in data, default order:
# 0	Planning Horizon (T)
# 1	Max. trips per period	
# 2	Max. pallets in area

model.T = pyo.Set(initialize=range(1, extraParam.iloc[0,1]+1), doc="Set of days")
model.M = pyo.Set(initialize=range(1, extraParam.iloc[1,1]+1), doc="Set of possible trips per day")

model.S = pyo.Set(initialize=range(1, pallets["Pallet Size"].max()+1), doc="Set of pallet sizes")
model.O = pyo.Set(initialize=orders["Order ID"].unique() ,doc="Set of orders")
# model.J = pyo.Set(initialize=range(1, int(sorted(pallets["Product Type"].unique())[-1][-1])+1), doc="Set of products") 
model.J = pyo.Set(initialize=pallets["Product Type"].unique(), doc="Set of products") 

model.I.display(), model.K.display(), model.T.display(), model.M.display(), model.S.display(), model.O.display(), model.J.display()

I : Set of pallets
    Size=1, Index=None, Ordered=Insertion
    Key  : Dimen : Domain : Size : Members
    None :     1 :    Any :  436 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194

(None, None, None, None, None, None, None)

In [45]:
capacity_dict = {}
for row in trucks["Vehicle Type"].unique():
    row=trucks[trucks["Vehicle Type"]==row].iloc[0,:]
    k=int(row.iloc[1])
    s=1
    capacity_dict[(k, s)] = row.iloc[2]
    k=int(row.iloc[1])
    s=2
    capacity_dict[(k, s)] = row.iloc[3]

demand_dict = {}
for row in orders.iterrows():
    o=row[1].iloc[0]
    j=row[1].iloc[1]
    demand_dict[(o, j)] = row[1].iloc[2]

In [47]:
# --------------------------------------------------------
# 3) Define Parameters
# --------------------------------------------------------
model.release_day = pyo.Param(model.I, initialize=dict(enumerate(pallets["Release Day"], start=1)), within=pyo.NonNegativeIntegers, doc="r_i: Release day for pallet i")
model.c_owned     = pyo.Param(model.K, initialize=dict(enumerate(trucks.groupby(["Vehicle Type","Fixed Cost (c_k)"]).count().reset_index().iloc[:,1], start=1)), within=pyo.NonNegativeReals, doc="c_k: Cost of using an owned vehicle type k")
model.c_rented    = pyo.Param(model.K, initialize=dict(enumerate(trucks.groupby(["Vehicle Type","Variable Cost (c'_k)"]).count().reset_index().iloc[:,1], start=1)), within=pyo.NonNegativeReals, doc="c'_k: Cost of renting vehicle type k")

#!!!!!!! Unchecked initialization
model.h           = pyo.Param(model.O, initialize=orders.groupby(["Order ID","Earliness Penalty"]).count().reset_index().iloc[:,0:2].set_index("Order ID").to_dict()["Earliness Penalty"], within=pyo.NonNegativeReals, doc="h_o: Per-day earliness penalty for order o")
model.d_due       = pyo.Param(model.O, initialize=orders.groupby(["Order ID","Due Date"]).count().reset_index().iloc[:,0:2].set_index("Order ID").to_dict()["Due Date"], within=pyo.NonNegativeIntegers, doc="d_o: Due day for order o")


model.p_demand    = pyo.Param(model.O, model.J, initialize=demand_dict, default=0, within=pyo.NonNegativeIntegers, doc="p_{o,j}: Demand for product j in order o")
demand_dict=None

model.j_of_i      = pyo.Param(model.I, initialize=dict(enumerate(pallets["Product Type"], start=1)), within=model.J, doc="j(i): Product type of pallet i")
model.n_i         = pyo.Param(model.I, initialize=dict(enumerate(pallets["Amount"], start=1)), within=pyo.NonNegativeIntegers, doc="n_i: Product capacity of pallet i")


model.capacity    = pyo.Param(model.K, model.S, initialize=capacity_dict, within=pyo.NonNegativeIntegers, doc="Capacity_{k,s}: # of pallets of size s that fit in vehicle k")
capacity_dict=None

model.b_k         = pyo.Param(model.K, initialize=dict(enumerate(trucks.groupby("Vehicle Type").count().reset_index().iloc[:,1], start=1)), within=pyo.NonNegativeIntegers, doc="b_k: Number of owned vehicles of type k available")

model.q           = pyo.Param(initialize=extraParam.iloc[2,1],within=pyo.NonNegativeIntegers, doc="q: Max number of pallets allowed in waiting area overnight")

model.release_day.display(), model.h.display(), model.d_due.display(), model.p_demand.display(), model.j_of_i.display(), model.n_i.display(), model.capacity.display(), model.q.display()

release_day : r_i: Release day for pallet i
    Size=436, Index=I, Domain=NonNegativeIntegers, Default=None, Mutable=False
    Key : Value
      1 :     1
      2 :     1
      3 :     1
      4 :     1
      5 :     1
      6 :     1
      7 :     1
      8 :     1
      9 :     1
     10 :     1
     11 :     1
     12 :     1
     13 :     1
     14 :     1
     15 :     1
     16 :     1
     17 :     1
     18 :     1
     19 :     1
     20 :     1
     21 :     1
     22 :     1
     23 :     1
     24 :     1
     25 :     1
     26 :     1
     27 :     1
     28 :     1
     29 :     1
     30 :     1
     31 :     1
     32 :     1
     33 :     1
     34 :     1
     35 :     1
     36 :     1
     37 :     2
     38 :     2
     39 :     2
     40 :     2
     41 :     2
     42 :     2
     43 :     2
     44 :     2
     45 :     2
     46 :     2
     47 :     2
     48 :     2
     49 :     2
     50 :     2
     51 :     2
     52 :     2
     53 :     2
     54 :    

(None, None, None, None, None, None, None, None)

In [48]:

# --------------------------------------------------------
# 4) Define Decision Variables
# --------------------------------------------------------
# 1) Shipment Variables
model.u = pyo.Var(model.K, model.T, model.M, model.S, within=pyo.Binary,
                  doc="u_{k,t,m,s}: 1 if a shipment of vehicle type k on day t, trip m, uses pallet size s")
model.v = pyo.Var(model.K, model.T, model.M, model.S, within=pyo.NonNegativeIntegers,
                  doc="v_{k,t,m,s}: Number of pallets of size s in shipment (k,t,m)")

# 2) Vehicle Usage
model.y = pyo.Var(model.K, model.T, model.M, within=pyo.Binary,
                  doc="y_{k,t,m}: 1 if an owned vehicle of type k is used on day t, trip m")
model.z = pyo.Var(model.K, model.T, model.M, within=pyo.Binary,
                  doc="z_{k,t,m}: 1 if a rented vehicle of type k is used on day t, trip m")

# 3) Pallet Assignment
model.x = pyo.Var(model.I, model.K, model.T, model.M, model.S, within=pyo.Binary,
                  doc="x_{i,k,t,m,s}: 1 if pallet i is shipped via (k,t,m,s)")

# 4) Order Fulfillment
model.e = pyo.Var(model.I, model.O, model.T, within=pyo.NonNegativeIntegers,
                  doc="e_{i,o,t}: products from pallet i allocated to order o and shipped on day t")


model.e_sub= pyo.Var(model.I, model.O, model.T, within=pyo.Binary,
                  doc="e_sub_{i,o,t}: if any products from pallet i allocated to order o and shipped on day t")



# After declaring model.x, fix variables where t < earliest_day[i]
fixedCount=0
for i in model.I:
    T_i = model.release_day[i]
    for t in model.T:
        if t < T_i:
            for k in model.K:
                for m in model.M:
                    for s in model.S:
                        model.x[i, k, t, m, s].fix(0)
                        fixedCount+=1

print(fixedCount, "variables fixed to 0")

fixedCountE=0
for i in model.I:
    T_i = model.release_day[i]
    for t in model.T:
        if t < T_i:
            for o in model.O:
                model.e[i, o, t].fix(0)
                model.e_sub[i, o, t].fix(0)
                fixedCountE+=1


print(fixedCountE*2, "e variables fixed to 0")
# --------------------------------------------------------
# 5) Define Constraints
# --------------------------------------------------------

# (1) Each pallet is shipped exactly once, after its release day
def single_shipment_rule(model, i):
    return sum(model.x[i, k, t, m, s]
               for k in model.K
               for t in model.T if t >= model.release_day[i]
               for m in model.M
               for s in model.S) == 1
model.single_shipment = pyo.Constraint(model.I, rule=single_shipment_rule)


# (2) Shipment Capacity:
#     sum_i x_{i,k,t,m,s} = v_{k,t,m,s} <= capacity_{k,s} * u_{k,t,m,s}
# We can express this in two separate constraints:
def capacity_link_1_rule(model, k, t, m, s):
    # sum of pallets assigned = v_{k,t,m,s}
    return sum(model.x[i, k, t, m, s] for i in model.I) == model.v[k, t, m, s]
model.capacity_link_1 = pyo.Constraint(model.K, model.T, model.M, model.S, rule=capacity_link_1_rule)

def capacity_link_2_rule(model, k, t, m, s):
    # v_{k,t,m,s} <= capacity_{k,s} * u_{k,t,m,s}
    return model.v[k, t, m, s] <= model.capacity[k, s] * model.u[k, t, m, s]
model.capacity_link_2 = pyo.Constraint(model.K, model.T, model.M, model.S, rule=capacity_link_2_rule)


# (3) Single pallet size per shipment
def single_size_rule(model, k, t, m):
    return sum(model.u[k, t, m, s] for s in model.S) <= 1
model.single_size = pyo.Constraint(model.K, model.T, model.M, rule=single_size_rule)


# (4) Vehicle Type usage constraint
#     y_{k,t,m} + z_{k,t,m} = sum_s u_{k,t,m,s}
def vehicle_type_rule(model, k, t, m):
    return model.y[k, t, m] + model.z[k, t, m] == sum(model.u[k, t, m, s] for s in model.S)
model.vehicle_type = pyo.Constraint(model.K, model.T, model.M, rule=vehicle_type_rule)


# (5) Owned Vehicle Daily Trip Limit
#     sum_m y_{k,t,m} <= 3 * b_k
# Here we assume each type k can do up to 3 trips per owned vehicle. Adapt if needed.
def owned_vehicle_limit_rule(model, k, t):
    return sum(model.y[k, t, m] for m in model.M) <= 3 * model.b_k[k]
model.owned_vehicle_limit = pyo.Constraint(model.K, model.T, rule=owned_vehicle_limit_rule)


# (6) Order Demand:
#     sum_{i: j(i)=j} sum_{t <= d_o} e_{i,o,t} >= p_{o,j}
# We iterate over each order o and each product j
# If p_demand[o,j] > 0, we must satisfy that demand by the due date.
def order_demand_rule(model, o, j):
    # sum of allocated products across all pallets i of type j
    return sum(model.e[i, o, t]
               for i in model.I
               if model.j_of_i[i] == j
               for t in model.T
               if t <= model.d_due[o]) >= model.p_demand[o, j]
model.order_demand = pyo.Constraint(model.O, model.J, rule=order_demand_rule)


# (7) Product allocation per pallet:
#     sum_o e_{i,o,t} <= n_i * sum_{k,m,s} x_{i,k,t,m,s}
def product_alloc_rule(model, i, t):
    return sum(model.e[i, o, t] for o in model.O) <= \
           model.n_i[i] * sum(model.x[i, k, t, m, s] for k in model.K for m in model.M for s in model.S)
model.product_alloc = pyo.Constraint(model.I, model.T, rule=product_alloc_rule)


# (8) Waiting Area Limit:
#     For each day t, the number of pallets that have been released but not shipped by day t cannot exceed q
#     sum_{i: r_i <= t} (1 - sum_{t' <= t, k,m,s} x_{i,k,t',m,s}) <= q
def waiting_area_rule(model, t):
    # Only consider pallets i whose release_day[i] <= t
    return sum(
        1 - sum(model.x[i, k, t_prime, m, s]
                for t_prime in model.T if t_prime <= t
                for k in model.K
                for m in model.M
                for s in model.S)
        for i in model.I
        if model.release_day[i] <= t
    ) <= model.q
model.waiting_area = pyo.Constraint(model.T, rule=waiting_area_rule)


M = 300

def e_sub_rule(model, i, o, t):
    # If any products from pallet i are allocated to order o on day t, then e_sub[i,o,t] must be 1.
    return model.e[i, o, t] <= M * model.e_sub[i, o, t]
model.e_sub_constraint = pyo.Constraint(model.I, model.O, model.T, rule=e_sub_rule)

# --------------------------------------------------------
# 6) Define Objective
# --------------------------------------------------------
# Minimize:
#   sum_{k,t,m} (c_k * y_{k,t,m} + c'_k * z_{k,t,m})
#   + sum_{i,o,t} (d_o - t)*h_o * e_{i,o,t}
def objective_rule(model):
    vehicle_cost = sum(model.c_owned[k]*model.y[k, t, m] + model.c_rented[k]*model.z[k, t, m]
                       for k in model.K for t in model.T for m in model.M)
    earliness_cost = sum((model.d_due[o] - t)*model.h[o]*model.e_sub[i, o, t]
                         for i in model.I
                         for o in model.O
                         for t in model.T
                         # typically you'd restrict to t <= d_due[o] if you only penalize earliness
                         if t <= model.d_due[o])
    return vehicle_cost + earliness_cost

model.Obj = pyo.Objective(rule=objective_rule, sense=pyo.minimize)


22824 variables fixed to 0
111584 e variables fixed to 0


In [49]:
import time
#edanın commitinde kodu ben (Yasin) yazdım

#Main attempt to add checkpoints but it is not working
def solve_with_checkpoints(model, max_hours=10, checkpoint_interval=1):
    """
    Solve the model in repeated time blocks (checkpoint_interval hours),
    saving the best solution so far (i.e., a 'checkpoint') each time
    one block finishes. Continue until either the model is solved optimally
    or until max_hours is reached.
    """
    solver = pyo.SolverFactory('gurobi')
    #cbc 

    # Convert hours to seconds
    one_block_seconds = checkpoint_interval * 300
    hours_elapsed = 0
    iteration = 0

    # Loop until we've run out of time or found an optimal solution
    while hours_elapsed < max_hours:
        # Instruct Gurobi to run for up to `checkpoint_interval` hours
        solver.options['TimeLimit'] = one_block_seconds

        # Use 'warm start' so that we pick up from the best known solution
        results = solver.solve(model, tee=True, warmstart=True)
        results.write()

        # Store the solution in the Pyomo model object
        model.solutions.load_from(results)

        # Save a checkpoint file of the best solution found so far
        iteration += 1
        timestamp = time.strftime("%Y%m%d_%H%M%S")
        checkpoint_name = f"checkpoint_{iteration}_{timestamp}.json"
        model.solutions.store_to(checkpoint_name)
        print(f"Checkpoint saved to {checkpoint_name}")

        # If the solver status is "ok" and the termination condition is "optimal",
        # we've found an optimal solution – no need to keep going.
        if (
            results.solver.status == pyo.SolverStatus.ok and
            results.solver.termination_condition == pyo.TerminationCondition.optimal
        ):
            print("Optimal solution found.")
            break

        # If we're here, the solver either hasn’t proven optimality yet
        # or ended on another condition – move on to the next time block
        hours_elapsed += checkpoint_interval

    return results


# # Example usage in your script:

# # Assuming you have already built your model and sets: model.I, model.K, model.T, model.M, model.S, etc.
# results = solve_with_checkpoints(model, max_hours=6, checkpoint_interval=1)

# # Now that solve() has finished (either found optimal or hit max time),
# # you can safely query the final/best known solution:
# for i in model.I:
#     for k in model.K:
#         for t in model.T:
#             for m in model.M:
#                 for s in model.S:
#                     if pyo.value(model.x[i, k, t, m, s]) > 0.5:
#                         print(f"Pallet {i} shipped by vehicle {k} on day {t}, trip {m}, size {s}")



In [50]:
solver= pyo.SolverFactory('gurobi')
solver.options['TimeLimit'] = 300
solver.options["Threads"]=16


results = solver.solve(model, tee=True)
results.write()

# At this point, you can query solution values:
for i in model.I:
    for k in model.K:
        for t in model.T:
            for m in model.M:
                for s in model.S:
                    if pyo.value(model.x[i,k,t,m,s]) != 0:
                        print(f"Pallet {i} shipped by vehicle {k} on day {t}, trip {m}, size {s}")

Read LP format model from file C:\Users\alt_user\AppData\Local\Temp\tmpz4xmkj4r.pyomo.lp
Reading time = 0.38 seconds
x1: 138622 rows, 189483 columns, 454335 nonzeros
Set parameter TimeLimit to value 300
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-12700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Non-default parameters:
TimeLimit  300

Optimize a model with 138622 rows, 189483 columns and 454335 nonzeros
Model fingerprint: 0xe650223b
Variable types: 1 continuous, 189482 integer (110860 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [1e+01, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+02]
Presolve removed 126666 rows and 132875 columns
Presolve time: 0.08s
Presolved: 11956 rows, 56608 columns, 237128 nonzeros
Variable types: 0 continuous, 56608 integer (41540 binary)
Performing

In [51]:
sum(model.c_owned[k]*pyo.value(model.y[k, t, m])
                       for k in model.K for t in model.T for m in model.M)

1510.0

In [52]:
sum((model.d_due[o] - t)*model.h[o]*pyo.value(model.e_sub[i,o,t])
                         for i in model.I
                         for o in model.O
                         for t in model.T
                         # typically you'd restrict to t <= d_due[o] if you only penalize earliness
                         if t <= model.d_due[o])

616.0

In [53]:
for i in model.I:
    for o in model.O:
        for t in model.T:
                    if pyo.value(model.e[i,o,t]) != 0:
                        print(f"Pallet {i} shipped to order {o} on day {t}")
                        # print(f"Pallet {i} shipped by vehicle {k} on day {t}, trip {m}, size {s}")

Pallet 1 shipped to order 8 on day 1
Pallet 1 shipped to order 20 on day 1
Pallet 2 shipped to order 15 on day 1
Pallet 3 shipped to order 15 on day 1
Pallet 4 shipped to order 8 on day 1
Pallet 4 shipped to order 15 on day 1
Pallet 5 shipped to order 8 on day 1
Pallet 5 shipped to order 27 on day 1
Pallet 6 shipped to order 8 on day 1
Pallet 6 shipped to order 21 on day 1
Pallet 7 shipped to order 8 on day 1
Pallet 8 shipped to order 21 on day 1
Pallet 9 shipped to order 7 on day 1
Pallet 9 shipped to order 15 on day 1
Pallet 10 shipped to order 8 on day 1
Pallet 11 shipped to order 15 on day 1
Pallet 11 shipped to order 19 on day 1
Pallet 12 shipped to order 8 on day 1
Pallet 12 shipped to order 19 on day 1
Pallet 13 shipped to order 21 on day 1
Pallet 14 shipped to order 21 on day 1
Pallet 15 shipped to order 21 on day 1
Pallet 16 shipped to order 8 on day 1
Pallet 16 shipped to order 15 on day 1
Pallet 17 shipped to order 15 on day 1
Pallet 18 shipped to order 8 on day 1
Pallet 18 

In [54]:
# At this point, you can query solution values:
sumY=0
sumZ=0
sumE=0
for k in model.K:
    for t in model.T:
        for m in model.M:
                # print(f"{pyo.value(model.y[k,t,m])}")
                sumY+=pyo.value(model.y[k,t,m])
                sumZ+=pyo.value(model.z[k,t,m])
                sumE+=pyo.value(model.e[i, o, t])
                # if pyo.value(model.y[k,t,m]) > 0.5:
                    # print(f"Pallet {i} shipped by vehicle {k} on day {t}, trip {m}, size {s}")


for i in model.I:
    for o in model.O:
        for t in model.T:
            sumE+=pyo.value(model.e[i, o, t])      

In [55]:
sumY,sumZ,sumE

(16.0, 0.0, 32570.0)